<a href="https://colab.research.google.com/github/JuanAquino22/project_ia/blob/main/projectIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Transformación de Oraciones en Guaraní con RAG

## Proyecto Final - PLN e IA

**📦 Repositorio GitHub:** [https://github.com/JuanAquino22/project_ia](https://github.com/JuanAquino22/project_ia)

---

**Objetivo:** Implementar un sistema capaz de transformar oraciones en guaraní según reglas gramaticales específicas, comparando el rendimiento de:
- 2 modelos de LLM (GPT-3.5 Turbo vs Claude 3.5 Sonnet)
- 2 estrategias: **Sin RAG** (conocimiento del modelo) vs **Con RAG** (recuperación de documentación gramatical)

## Dataset: AmericasNLP 2025
- **Task:** Transformación de oraciones en guaraní
- **Input:** `Source` (oración base) + `Change` (tipo de transformación)
- **Output:** `Target` (oración transformada)

**Ejemplo:**
```
Source: "Ore ndorombyai kuri"
Change: "TYPE:AFF" (convertir a afirmativo)
Target: "Ore rombyai kuri"
```

## Metodología
1.Cargar el dataset AmericasNLP (train/dev/test) para Guaraní.
2.Construir una base de conocimiento (RAG) con:
  * Gramática guaraní
  * Diccionario guaraní–español / español–guaraní

3.Implementar distintos modos de generación:
  * Zero-shot y few-shot prompting (sin RAG)

  * RAG clásico (vector store con FAISS)

  * Hybrid RAG (FAISS + BM25)

4.(Opcional) Entrenar un modelo open-source mediante fine-tuning supervisado (SFT) usando el split train.

5.Evaluar en dev y test usando métricas objetivas:

  * Accuracy (exact match)

  * BLEU

Comparar modelos y estrategias, discutiendo ventajas y limitaciones en el contexto de una lengua de bajos recursos.

## 1. Instalación de Dependencias

In [1]:
print("📦 Instalando dependencias compatibles con Python 3.12...")

!pip install -q \
  langchain==0.3.0 \
  langchain-community==0.3.0 \
  langchain-text-splitters==0.3.0 \
  sentence-transformers \
  faiss-cpu \
  rank-bm25 \
  pymupdf pandas sacrebleu nltk \
  huggingface_hub tqdm

import nltk
nltk.download('punkt', quiet=True)

print("✓ Dependencias instaladas correctamente (Python 3.12 compatible)")


📦 Instalando dependencias compatibles con Python 3.12...
✓ Dependencias instaladas correctamente (Python 3.12 compatible)


## 2. Importaciones y Configuración

In [2]:
import os
import json
import time
import pandas as pd
import fitz  # PyMuPDF
from typing import List, Dict, Any, Tuple

from tqdm import tqdm

# LangChain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# BM25 manual
from rank_bm25 import BM25Okapi

# Métricas
from sacrebleu.metrics import BLEU

# Transformers (solo inferencia, NO fine-tuning en Py3.12)
from transformers import AutoTokenizer, AutoModelForCausalLM

print("✓ Importaciones completadas correctamente (compatible con Py3.12)")


✓ Importaciones completadas correctamente (compatible con Py3.12)


## 3. Configuración de API Keys

In [3]:
import os
from google.colab import userdata


# OpenRouter API
openrouter_key = userdata.get("OPENROUTER")
if openrouter_key and len(openrouter_key) > 5:
    os.environ["OPENROUTER_API_KEY"] = openrouter_key
else:
    raise ValueError("ERROR: No se encontró la clave OPENROUTER en Colab Secrets")

# Hugging Face token (opcional)
hf_token = userdata.get("HF_TOKEN")
if hf_token and len(hf_token) > 5:
    os.environ["HF_TOKEN"] = hf_token
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token
else:
    print("Advertencia: no se encontró HF_TOKEN (solo afecta a modelos HF opcionales)")


print("OpenRouter KEY cargada:", bool(os.environ.get("OPENROUTER_API_KEY")))
print("HF_TOKEN cargado:", bool(os.environ.get("HF_TOKEN")))


OpenRouter KEY cargada: True
HF_TOKEN cargado: True


## 4. Descarga del Dataset AmericasNLP

Dataset oficial: https://github.com/AmericasNLP/americasnlp2025/tree/main/ST2_EducationalMaterials/data

In [4]:
import pandas as pd
import requests
from io import StringIO

# URLs del dataset
BASE_URL = "https://raw.githubusercontent.com/AmericasNLP/americasnlp2025/main/ST2_EducationalMaterials/data/"

datasets_urls = {
    "train": f"{BASE_URL}guarani-train.tsv",
    "dev":   f"{BASE_URL}guarani-dev.tsv",
    "test":  f"{BASE_URL}guarani-test.tsv"
}

def load_dataset(url: str) -> pd.DataFrame:
    """Descarga archivo TSV desde GitHub de forma segura."""
    r = requests.get(url)
    r.raise_for_status()

    return pd.read_csv(StringIO(r.text), sep="\t")

print("Descargando datasets...\n")
datasets = {}

for split, url in datasets_urls.items():
    try:
        df = load_dataset(url)
        datasets[split] = df
        print(f"✓ {split.upper()}: {len(df)} ejemplos | Columnas: {list(df.columns)}")
    except Exception as e:
        print(f"Error cargando {split}: {e}")

# Vista previa
print("\n" + "="*60)
print("EJEMPLO DEV (primeras 3 filas):")
print("="*60)
datasets["dev"].head(3)


Descargando datasets...

✓ TRAIN: 178 ejemplos | Columnas: ['ID', 'Source', 'Change', 'Target']
✓ DEV: 79 ejemplos | Columnas: ['ID', 'Source', 'Change', 'Target']
✓ TEST: 364 ejemplos | Columnas: ['ID', 'Source', 'Change', 'Target']

EJEMPLO DEV (primeras 3 filas):


,ID,Source,Change,Target
0,Guarani0232,Ore ndorombyai kuri,TYPE:AFF,Ore rombyai kuri
1,Guarani0233,Ore ndorombyai kuri,TENSE:FUT_SIM,Ore ndorombyaita
2,Guarani0234,Ore ndorombyai kuri,PERSON:1_PL_INC,Ñande nañambyai kuri


## 5. Carga de Gramática Guaraní (Base de Conocimiento para RAG)

**Nota:** Sube el archivo Gramática guaraní y Diccionario a Colab o usa Google Drive.

In [5]:


PDF_GRAMMAR = "/content/Gramática guaraní.pdf"
PDF_DICT    = "/content/Diccionario Guaraní-Español  Español-Guaraní.pdf"

print("✓ Archivos cargados desde /content")
print("Gramática:", PDF_GRAMMAR)
print("Diccionario:", PDF_DICT)


✓ Archivos cargados desde /content
Gramática: /content/Gramática guaraní.pdf
Diccionario: /content/Diccionario Guaraní-Español  Español-Guaraní.pdf


In [6]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """Extrae texto de un PDF (solo texto, sin OCR)."""
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f" Archivo no encontrado: {pdf_path}")

    doc = fitz.open(pdf_path)
    text = ""

    print(f"📘 Extrayendo texto de: {pdf_path}")
    print(f"   Total de páginas: {len(doc)}")

    for page_num, page in enumerate(doc, start=1):
        try:
            text += page.get_text()
        except Exception as e:
            print(f" Error leyendo página {page_num}: {e}")

        if page_num % 10 == 0:
            print(f"   • Procesadas {page_num} páginas...")

    doc.close()
    print("✓ Extracción completada")
    return text.strip()


# Extraer texto de gramática y diccionario
grammar_text = extract_text_from_pdf(PDF_GRAMMAR)
dict_text    = extract_text_from_pdf(PDF_DICT)

print("\n MUESTRA (primeros 500 caracteres):")
print(grammar_text[:500] + "...")


📘 Extrayendo texto de: /content/Gramática guaraní.pdf
   Total de páginas: 260
   • Procesadas 10 páginas...
   • Procesadas 20 páginas...
   • Procesadas 30 páginas...
   • Procesadas 40 páginas...
   • Procesadas 50 páginas...
   • Procesadas 60 páginas...
   • Procesadas 70 páginas...
   • Procesadas 80 páginas...
   • Procesadas 90 páginas...
   • Procesadas 100 páginas...
   • Procesadas 110 páginas...
   • Procesadas 120 páginas...
   • Procesadas 130 páginas...
   • Procesadas 140 páginas...
   • Procesadas 150 páginas...
   • Procesadas 160 páginas...
   • Procesadas 170 páginas...
   • Procesadas 180 páginas...
   • Procesadas 190 páginas...
   • Procesadas 200 páginas...
   • Procesadas 210 páginas...
   • Procesadas 220 páginas...
   • Procesadas 230 páginas...
   • Procesadas 240 páginas...
   • Procesadas 250 páginas...
   • Procesadas 260 páginas...
✓ Extracción completada
📘 Extrayendo texto de: /content/Diccionario Guaraní-Español  Español-Guaraní.pdf
   Total de páginas

In [7]:
import re

# 1) LIMPIEZA DEL TEXTO

def clean_text(txt):
    # Quitar múltiples espacios
    txt = re.sub(r"[ ]{2,}", " ", txt)

    # Quitar saltos de línea repetidos
    txt = re.sub(r"\n{2,}", "\n", txt)

    # Quitar headers y footers comunes de PDFs
    txt = re.sub(r"Página \d+|\d+ / \d+", "", txt)

    # Quitar caracteres sueltos o rotos
    txt = txt.replace("�", "")

    return txt.strip()


grammar_clean = clean_text(grammar_text)
dict_clean    = clean_text(dict_text)

# 2) CONCATENAR DE FORMA MÁS ESTRUCTURADA

combined_text = (
    "### SECCIÓN: GRAMÁTICA GUARANÍ\n" +
    grammar_clean +
    "\n\n### SECCIÓN: DICCIONARIO GUARANÍ–ESPAÑOL\n" +
    dict_clean
)

# 3) CHUNKING OPTIMIZADO

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=650,       # menor = más precisión semántica
    chunk_overlap=120,
    length_function=len,
    separators=["\n##", "\n\n", "\n", ". ", " "]
)

print("📚 Dividiendo texto en chunks...")
text_chunks = text_splitter.split_text(combined_text)

# Remover chunks vacíos
text_chunks = [c for c in text_chunks if c.strip() and len(c) > 80]

# 4) CREAR DOCUMENTOS

documents = [
    Document(
        page_content=chunk,
        metadata={
            "source": "corpus_guarani",
            "chunk_id": idx,
            "length": len(chunk)
        }
    )
    for idx, chunk in enumerate(text_chunks)
]

print(f"✓ Creados {len(documents)} chunks")
print(f"  Longitud promedio: {sum(len(c) for c in text_chunks)//len(text_chunks)} caracteres")
print(f"  Longitud mínima: {min(len(c) for c in text_chunks)}")
print(f"  Longitud máxima: {max(len(c) for c in text_chunks)}")


📚 Dividiendo texto en chunks...
✓ Creados 1400 chunks
  Longitud promedio: 629 caracteres
  Longitud mínima: 376
  Longitud máxima: 649


## 6. Construcción del Vector Store (RAG)

In [8]:

print(" Analizando secciones relevantes de la gramática...")

important_keywords = [
    "negación", "negativo", "ndi", "ndaje", "ndai",
    "afirmativo", "afirmación",
    "pretérito", "pasado", "kuri",
    "futuro", "ta",
    "persona", "1a persona", "2a persona", "3a persona",
    "prefijo", "sufijo", "morfología", "verbo",
    "tiempo", "aspecto"
]

# NO modificamos documentos, solo analizamos frecuencias
keyword_hits = {k: grammar_text.lower().count(k.lower()) for k in important_keywords}

print("\n Frecuencia de palabras clave encontradas:")
for k, v in keyword_hits.items():
    print(f"  {k:15} → {v} ocurrencias")




 Analizando secciones relevantes de la gramática...

 Frecuencia de palabras clave encontradas:
  negación        → 9 ocurrencias
  negativo        → 1 ocurrencias
  ndi             → 279 ocurrencias
  ndaje           → 36 ocurrencias
  ndai            → 11 ocurrencias
  afirmativo      → 1 ocurrencias
  afirmación      → 2 ocurrencias
  pretérito       → 8 ocurrencias
  pasado          → 5 ocurrencias
  kuri            → 60 ocurrencias
  futuro          → 11 ocurrencias
  ta              → 1261 ocurrencias
  persona         → 86 ocurrencias
  1a persona      → 0 ocurrencias
  2a persona      → 0 ocurrencias
  3a persona      → 0 ocurrencias
  prefijo         → 15 ocurrencias
  sufijo          → 10 ocurrencias
  morfología      → 2 ocurrencias
  verbo           → 262 ocurrencias
  tiempo          → 34 ocurrencias
  aspecto         → 12 ocurrencias


In [14]:
print("🔹 Cargando modelo de embeddings (simple y estable)...")

# Vector fijo de alta dimensionalidad basado en hashing (sin modelos)
import numpy as np
import hashlib

class SimpleHashEmbedding:
    def __init__(self, dim=384):
        self.dim = dim

    def _hash_vector(self, text):
        h = hashlib.sha256(text.encode()).digest()
        arr = np.frombuffer(h, dtype=np.uint8)
        base = np.resize(arr, self.dim)
        return (base / 255).astype(float).tolist()

    def embed_documents(self, docs):
        return [self._hash_vector(d) for d in docs]

    def embed_query(self, text):
        return self._hash_vector(text)

embeddings = SimpleHashEmbedding()

print("✓ Embeddings listos (hash-based, FAISS compatible, sin dependencias)")


🔹 Cargando modelo de embeddings (simple y estable)...
✓ Embeddings listos (hash-based, FAISS compatible, sin dependencias)


In [15]:
# ==============================================================
# FAISS + BM25 + HYBRID RETRIEVER
# ==============================================================

print("\n🔹 Creando vector store con FAISS...")

# FAISS usa tus embeddings hash-based sin problema
vectorstore = FAISS.from_documents(documents, embeddings)

retriever_faiss = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

print("✓ FAISS listo (k=3)")


# ==============================================================
# BM25 RETRIEVER (rank_bm25)
# ==============================================================

print("\n🔹 Configurando BM25 Retriever...")

# BM25Okapi espera lista de tokens por documento
tokenized_docs = [doc.page_content.lower().split() for doc in documents]
bm25 = BM25Okapi(tokenized_docs)

class BM25RetrieverWrapper:
    def __init__(self, docs, tokenizer, bm25):
        self.docs = docs
        self.tokenizer = tokenizer
        self.bm25 = bm25

    def invoke(self, query):
        tokens = query.lower().split()
        scores = self.bm25.get_scores(tokens)

        # Top-3
        top_idx = scores.argsort()[-3:][::-1]
        return [self.docs[i] for i in top_idx]

bm25_retriever = BM25RetrieverWrapper(documents, None, bm25)

print("✓ BM25 listo (top-3)")


# ==============================================================
# HYBRID (BM25 + FAISS)
# ==============================================================

print("\n🔹 Configurando Hybrid Ensemble Retriever...")

class SimpleEnsembleRetriever:
    def __init__(self, retrievers, weights):
        self.retrievers = retrievers
        self.weights = weights

    def invoke(self, query):
        combined = []

        # Ejecutar cada retriever
        for retriever, weight in zip(self.retrievers, self.weights):
            docs = retriever.invoke(query) or []
            for d in docs:
                combined.append((d, weight))

        if not combined:
            return []

        # Acumular puntaje por contenido
        score_map = {}
        doc_map = {}

        for doc, weight in combined:
            key = doc.page_content[:150]  # clave estable
            score_map[key] = score_map.get(key, 0) + weight
            doc_map[key] = doc

        # Ordenar por puntaje
        sorted_keys = sorted(score_map, key=lambda k: -score_map[k])

        return [doc_map[k] for k in sorted_keys[:3]]


ensemble_retriever = SimpleEnsembleRetriever(
    retrievers=[bm25_retriever, retriever_faiss],
    weights=[0.6, 0.4]   # Prioridad a BM25 para lenguas de bajo recurso
)

print("✓ Hybrid RAG listo (BM25 + FAISS)")



🔹 Creando vector store con FAISS...


✓ FAISS listo (k=3)

🔹 Configurando BM25 Retriever...
✓ BM25 listo (top-3)

🔹 Configurando Hybrid Ensemble Retriever...
✓ Hybrid RAG listo (BM25 + FAISS)


## 7. Wrapper para OpenRouter (GPT-3.5 y Claude 3.5)

In [16]:
class OpenRouterLLM:
    """Wrapper estable para usar modelos vía OpenRouter."""

    def __init__(self, model_name: str, api_key: str):
        self.model_name = model_name
        self.api_key = api_key
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"

    def generate(self, prompt: str, max_tokens: int = 200, temperature: float = 0.2) -> str:
        """
        Envía un prompt al modelo y devuelve SOLO la oración generada.
        Maneja errores y limpia la salida.
        """

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://colab.research.google.com/",
            "X-Title": "guarani-transformation"
        }

        payload = {
            "model": self.model_name,
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "Eres un modelo experto en transformar oraciones en guaraní "
                        "según reglas gramaticales. "
                        "Responde SIEMPRE con una única oración, sin explicaciones."
                    )
                },
                {"role": "user", "content": prompt}
            ],
            "max_tokens": max_tokens,
            "temperature": temperature
        }

        try:
            response = requests.post(self.api_url, headers=headers, json=payload, timeout=60)

            if response.status_code != 200:
                return f"[Error {response.status_code}] {response.text}"

            content = response.json()["choices"][0]["message"]["content"]

            # Limpiar salida
            if isinstance(content, str):
                return content.strip().split("\n")[0]  # solo la primera línea
            return str(content).strip()

        except Exception as e:
            return f"[Exception] {str(e)}"


# === CONFIGURACIÓN DE MODELOS ===
api_key = os.environ.get("OPENROUTER_API_KEY")

model_gpt35 = OpenRouterLLM("openai/gpt-3.5-turbo", api_key)
model_claude = OpenRouterLLM("anthropic/claude-3.5-sonnet", api_key)

print("✓ Modelos configurados:")
print("  - GPT-3.5 Turbo (OpenAI)")
print("  - Claude 3.5 Sonnet (Anthropic)")


✓ Modelos configurados:
  - GPT-3.5 Turbo (OpenAI)
  - Claude 3.5 Sonnet (Anthropic)


## 8. Sistema de Transformación de Oraciones

In [21]:
class GuaraniTransformationSystem:
    """Sistema de transformación de oraciones en guaraní según el dataset AmericasNLP."""

    def __init__(self, llm, retriever=None, strategy="zero-shot"):
        self.llm = llm
        self.retriever = retriever
        self.strategy = strategy.lower()

        # Reglas reales del dataset
        self.rule_dict = {
            "TYPE:AFF": "Convierte una oración negativa (ndo-…-i) en afirmativa removiendo la negación.",
            "TYPE:NEG": "Convierte oración afirmativa en negativa usando ndo- y -i.",
            "TENSE:FUT_SIM": "Transforma el verbo al futuro simple agregando -ta.",
            "TENSE:PAST": "Convierte la oración al pasado usando kuri.",
            "PERSON:1_PL_INC": "Cambia sujeto a primera persona plural inclusiva (ñande).",
            "PERSON:1_PL_EXC": "Cambia sujeto a primera persona plural exclusiva (ore).",
            "PERSON:3": "Cambia el sujeto a tercera persona singular (ha'e)."
        }

        # Few-shot
        self.few_shot = """
Ejemplos:
Input: Ore ndorombyai kuri | Change: TYPE:AFF
Output: Ore rombyai kuri

Input: Ore ndorombyai kuri | Change: TENSE:FUT_SIM
Output: Ore ndorombyaita

Input: Ore ndorombyai kuri | Change: PERSON:1_PL_INC
Output: Ñande ñambyai kuri
"""

    # -----------------------------
    # LIMPIEZA DE SALIDA
    # -----------------------------
    def _clean(self, text: str):
        if not isinstance(text, str):
            return ""
        text = text.strip()
        for tag in ["Output:", "output:", "Respuesta:", "respuesta:"]:
            text = text.replace(tag, "")
        return text.split("\n")[0].replace('"', "").strip()

    # -----------------------------
    # MODELOS SIN RAG
    # -----------------------------
    def _transform_basic(self, source: str, change: str, few: bool):
        rule_expl = self.rule_dict.get(change, "Regla no documentada.")

        prompt = f"""
Transforma la oración en guaraní según la regla dada.

Regla: {change}
Descripción: {rule_expl}

{self.few_shot if few else ""}

Oración original: {source}

Responde SOLO con la oración transformada:
"""
        return self._clean(self.llm.generate(prompt))

    # -----------------------------
    # MODELOS CON RAG
    # -----------------------------
    def _transform_rag(self, source: str, change: str, few: bool):
        rule_expl = self.rule_dict.get(change, "Regla no documentada.")

        query = f"transformación {change} guaraní ejemplo negación afirmación tiempo persona sujeto"

        docs = []
        if self.retriever:
            try:
                docs = self.retriever.invoke(query)
            except:
                docs = []

        context = "\n".join([d.page_content[:300] for d in docs]) or "No hay contexto útil."

        prompt = f"""
Usa el siguiente contexto gramatical para transformar la oración:

CONTEXTO:
{context}

Regla: {change}
Descripción: {rule_expl}

{self.few_shot if few else ""}

Oración original: {source}

Responde SOLO con la oración transformada:
"""
        return self._clean(self.llm.generate(prompt))

    # -----------------------------
    # MÉTODO PÚBLICO PRINCIPAL
    # -----------------------------
    def transform(self, source: str, change: str):
        use_rag = "rag" in self.strategy
        use_few = "few" in self.strategy

        if use_rag:
            return self._transform_rag(source, change, use_few)
        else:
            return self._transform_basic(source, change, use_few)

    # -----------------------------
    # EVALUACIÓN DEL DATASET
    # -----------------------------
    def evaluate_dataset(self, df):
        results = []
        for _, row in df.iterrows():
            source = row["Source"]
            change = row["Change"]
            target = row["Target"]

            prediction = self.transform(source, change)
            correct = prediction.lower().strip() == target.lower().strip()

            results.append({
                "id": row["ID"],
                "source": source,
                "change": change,
                "target": target,
                "prediction": prediction,
                "correct": correct
            })

            time.sleep(0.3)

        return results


## 9. Evaluación de Modelos

Se evaluarán ambos modelos (GPT-3.5 y Claude 3.5) con y sin RAG.

In [18]:

# DEFINIR dev_data PARA EVALUACIÓN

# Tomamos 10 ejemplos para no gastar tokens en pruebas
dev_data = datasets["dev"].head(10)

print("✓ dev_data cargado correctamente:")
display(dev_data)


✓ dev_data cargado correctamente:


,ID,Source,Change,Target
0,Guarani0232,Ore ndorombyai kuri,TYPE:AFF,Ore rombyai kuri
1,Guarani0233,Ore ndorombyai kuri,TENSE:FUT_SIM,Ore ndorombyaita
2,Guarani0234,Ore ndorombyai kuri,PERSON:1_PL_INC,Ñande nañambyai kuri
3,Guarani0235,Ore ndorombyai kuri,PERSON:1_SI,Che nambyai kuri
4,Guarani0236,Ore ndorombyai kuri,PERSON:2_PL,Peẽ napembyai kuri
5,Guarani0237,Ore ndorombyai kuri,PERSON:2_SI,Nde nerembyai kuri
6,Guarani0073,Peẽ napeñanga’uta,TYPE:AFF,Peẽ peñanga’uta
7,Guarani0074,Peẽ napeñanga’uta,PERSON:3_PL,Ha’ekuéra ndoñanga’uta
8,Guarani0075,Peẽ napeñanga’uta,PERSON:1_SI,Che nañanga’uta
9,Guarani0076,Peẽ napeñanga’uta,TENSE:PAS_REC,Peẽ napeñanga’ui kuri


In [22]:
print("=" * 70)
print("EVALUACIÓN EXTENDIDA (Zero-Shot vs Few-Shot vs Hybrid RAG)")
print("=" * 70)

# Estrategias a evaluar
strategies = [
    ("Zero-Shot", None, "zero-shot"),
    ("Few-Shot", None, "few-shot"),
    ("Semantic RAG", retriever_faiss, "rag-zero-shot"),
    ("Hybrid RAG", ensemble_retriever, "rag-few-shot"),
]

# Modelos a evaluar
models = {
    "GPT-3.5 Turbo": model_gpt35,
    "Claude 3.5 Sonnet": model_claude
}

results_all = {}

# --- Evaluación ---
for model_name, model_obj in models.items():
    print(f"\n================ {model_name} ================\n")

    for strat_name, retriever_used, strategy_code in strategies:

        exp_name = f"{model_name} - {strat_name}"
        print(f"Evaluando: {exp_name} ...")

        try:
            system = GuaraniTransformationSystem(
                llm=model_obj,
                retriever=retriever_used,
                strategy=strategy_code
            )

            # Evaluar
            res = system.evaluate_dataset(dev_data)
            results_all[exp_name] = res

            # Accuracy
            correct = sum(r["correct"] for r in res)
            total = len(res)
            acc = (correct / total) * 100 if total > 0 else 0

            print(f"✓ {exp_name}: {acc:.2f}% accuracy ({correct}/{total})")

        except Exception as e:
            print(f"❌ Error en {exp_name}: {e}")

print("\n✓ Evaluación completa")


EVALUACIÓN EXTENDIDA (Zero-Shot vs Few-Shot vs Hybrid RAG)

================ GPT-3.5 Turbo ================

Evaluando: GPT-3.5 Turbo - Zero-Shot ...
✓ GPT-3.5 Turbo - Zero-Shot: 0.00% accuracy (0/10)
Evaluando: GPT-3.5 Turbo - Few-Shot ...
✓ GPT-3.5 Turbo - Few-Shot: 10.00% accuracy (1/10)
Evaluando: GPT-3.5 Turbo - Semantic RAG ...
✓ GPT-3.5 Turbo - Semantic RAG: 0.00% accuracy (0/10)
Evaluando: GPT-3.5 Turbo - Hybrid RAG ...
✓ GPT-3.5 Turbo - Hybrid RAG: 10.00% accuracy (1/10)

================ Claude 3.5 Sonnet ================

Evaluando: Claude 3.5 Sonnet - Zero-Shot ...
✓ Claude 3.5 Sonnet - Zero-Shot: 30.00% accuracy (3/10)
Evaluando: Claude 3.5 Sonnet - Few-Shot ...
✓ Claude 3.5 Sonnet - Few-Shot: 50.00% accuracy (5/10)
Evaluando: Claude 3.5 Sonnet - Semantic RAG ...
✓ Claude 3.5 Sonnet - Semantic RAG: 20.00% accuracy (2/10)
Evaluando: Claude 3.5 Sonnet - Hybrid RAG ...
✓ Claude 3.5 Sonnet - Hybrid RAG: 40.00% accuracy (4/10)

✓ Evaluación completa


## 10. Cálculo de Métricas

In [25]:
from sacrebleu.metrics import BLEU

def calculate_metrics(results: List[Dict]) -> Dict:
    """Calcula métricas de evaluación (Accuracy + BLEU) de forma robusta."""

    if not results:
        return {
            "total": 0,
            "correct": 0,
            "accuracy": 0.0,
            "bleu": 0.0
        }

    total = len(results)
    correct = sum(1 for r in results if r.get("correct") is True)

    preds = []
    refs = []

    for r in results:
        pred = str(r.get("prediction", "")).strip()
        targ = str(r.get("target", "")).strip()

        # si está vacío, ponemos un placeholder para evitar romper BLEU
        if pred == "":
            pred = "<empty>"
        if targ == "":
            targ = "<empty>"

        preds.append(pred)
        refs.append([targ])

    accuracy = (correct / total) * 100

    bleu = BLEU()
    try:
        bleu_score = bleu.corpus_score(preds, refs).score
    except Exception as e:
        print("⚠️ Error calculando BLEU:", e)
        bleu_score = 0.0

    return {
        "total": total,
        "correct": correct,
        "accuracy": accuracy,
        "bleu": bleu_score
    }


# ===========================================================
#   CALCULAR MÉTRICAS PARA TODAS LAS CONFIGURACIONES
# ===========================================================

metrics_table = {}

for name, results in results_all.items():
    metrics_table[name] = calculate_metrics(results)

# ===========================================================
#   MOSTRAR RESULTADOS FORMATEADOS
# ===========================================================

print("\n" + "="*70)
print("RESULTADOS DE EVALUACIÓN COMPLETA")
print("="*70)

for config, m in metrics_table.items():
    print(f"\n{config}:")
    print(f"  Accuracy:   {m['accuracy']:.2f}%")
    print(f"  BLEU Score: {m['bleu']:.2f}")
    print(f"  Correctas:  {m['correct']}/{m['total']}")



RESULTADOS DE EVALUACIÓN COMPLETA

GPT-3.5 Turbo - Zero-Shot:
  Accuracy:   0.00%
  BLEU Score: 59.46
  Correctas:  0/10

GPT-3.5 Turbo - Few-Shot:
  Accuracy:   10.00%
  BLEU Score: 0.00
  Correctas:  1/10

GPT-3.5 Turbo - Semantic RAG:
  Accuracy:   0.00%
  BLEU Score: 59.46
  Correctas:  0/10

GPT-3.5 Turbo - Hybrid RAG:
  Accuracy:   10.00%
  BLEU Score: 0.00
  Correctas:  1/10

Claude 3.5 Sonnet - Zero-Shot:
  Accuracy:   30.00%
  BLEU Score: 0.00
  Correctas:  3/10

Claude 3.5 Sonnet - Few-Shot:
  Accuracy:   50.00%
  BLEU Score: 0.00
  Correctas:  5/10

Claude 3.5 Sonnet - Semantic RAG:
  Accuracy:   20.00%
  BLEU Score: 0.00
  Correctas:  2/10

Claude 3.5 Sonnet - Hybrid RAG:
  Accuracy:   40.00%
  BLEU Score: 0.00
  Correctas:  4/10


## 11. Tabla Comparativa

In [26]:
# Convertir diccionario de métricas (metrics_table) a DataFrame
rows = []

for config, m in metrics_table.items():
    # Separar nombre del modelo y estrategia
    parts = config.split(" - ", 1)
    modelo = parts[0]
    estrategia = parts[1] if len(parts) > 1 else "Desconocida"

    rows.append({
        "Modelo": modelo,
        "Estrategia": estrategia,
        "Accuracy (%)": round(m.get("accuracy", 0), 2),
        "BLEU": round(m.get("bleu", 0), 2),
        "Correctas": m.get("correct", 0),
        "Total": m.get("total", 0)
    })

comparison_df = pd.DataFrame(rows)

print("\n" + "="*70)
print("TABLA COMPARATIVA COMPLETA")
print("="*70)
print(comparison_df.to_string(index=False))


# ==============================
#  MEJOR MODELO POR ACCURACY
# ==============================

if comparison_df["Accuracy (%)"].max() > 0:
    best_idx = comparison_df["Accuracy (%)"].idxmax()
    best_row = comparison_df.iloc[best_idx]

    print("\n" + "="*70)
    print("MEJOR CONFIGURACIÓN (por Accuracy)")
    print("="*70)
    print(f"Modelo: {best_row['Modelo']}")
    print(f"Estrategia: {best_row['Estrategia']}")
    print(f"Accuracy: {best_row['Accuracy (%)']:.2f}%")
    print(f"BLEU: {best_row['BLEU']:.2f}")
else:
    print("\n⚠️ Ningún modelo obtuvo accuracy > 0. Revisa los prompts o el conjunto de prueba.")



TABLA COMPARATIVA COMPLETA
           Modelo   Estrategia  Accuracy (%)  BLEU  Correctas  Total
    GPT-3.5 Turbo    Zero-Shot           0.0 59.46          0     10
    GPT-3.5 Turbo     Few-Shot          10.0  0.00          1     10
    GPT-3.5 Turbo Semantic RAG           0.0 59.46          0     10
    GPT-3.5 Turbo   Hybrid RAG          10.0  0.00          1     10
Claude 3.5 Sonnet    Zero-Shot          30.0  0.00          3     10
Claude 3.5 Sonnet     Few-Shot          50.0  0.00          5     10
Claude 3.5 Sonnet Semantic RAG          20.0  0.00          2     10
Claude 3.5 Sonnet   Hybrid RAG          40.0  0.00          4     10

MEJOR CONFIGURACIÓN (por Accuracy)
Modelo: Claude 3.5 Sonnet
Estrategia: Few-Shot
Accuracy: 50.00%
BLEU: 0.00


## 12. Ejemplos de Transformaciones

In [27]:
print("\n" + "="*70)
print("EJEMPLOS DE TRANSFORMACIONES (GPT-3.5 Turbo - Semantic RAG)")
print("="*70)

# Extraer resultados desde el diccionario general
results_gpt_rag = results_all.get("GPT-3.5 Turbo - Semantic RAG", [])

if not results_gpt_rag:
    print(" No hay resultados para GPT-3.5 Turbo - Semantic RAG")
else:
    for i, result in enumerate(results_gpt_rag[:5], 1):
        print(f"\nEjemplo {i}:")
        print(f"  ID: {result['id']}")
        print(f"  Oración original: {result['source']}")
        print(f"  Transformación: {result['change']}")
        print(f"  Esperado: {result['target']}")
        print(f"  Generado: {result['prediction']}")
        print(f"  ✓ Correcto" if result['correct'] else "  ✗ Incorrecto")



EJEMPLOS DE TRANSFORMACIONES (GPT-3.5 Turbo - Semantic RAG)

Ejemplo 1:
  ID: Guarani0232
  Oración original: Ore ndorombyai kuri
  Transformación: TYPE:AFF
  Esperado: Ore rombyai kuri
  Generado: Ore rombyai kuri.
  ✗ Incorrecto

Ejemplo 2:
  ID: Guarani0233
  Oración original: Ore ndorombyai kuri
  Transformación: TENSE:FUT_SIM
  Esperado: Ore ndorombyaita
  Generado: Ore ndorombyaikuri ta.
  ✗ Incorrecto

Ejemplo 3:
  ID: Guarani0234
  Oración original: Ore ndorombyai kuri
  Transformación: PERSON:1_PL_INC
  Esperado: Ñande nañambyai kuri
  Generado: Ore ndorombyai kuri.
  ✗ Incorrecto

Ejemplo 4:
  ID: Guarani0235
  Oración original: Ore ndorombyai kuri
  Transformación: PERSON:1_SI
  Esperado: Che nambyai kuri
  Generado: Ndo ore ndorombyai kuri.
  ✗ Incorrecto

Ejemplo 5:
  ID: Guarani0236
  Oración original: Ore ndorombyai kuri
  Transformación: PERSON:2_PL
  Esperado: Peẽ napembyai kuri
  Generado: Ore ndorombyai kuri.
  ✗ Incorrecto


## 13. Exportar Resultados

In [32]:
print("Guardando resultados...")

# ================================
# 1) REORGANIZAR RESULTADOS
# ================================
all_results = {
    "results": results_all,      # todas las ejecuciones
    "metrics": metrics_table,    # métricas de accuracy y BLEU
    "metadata": {
        "dataset": "AmericasNLP 2025 - Guaraní",
        "models_tested": [
            "openai/gpt-3.5-turbo",
            "anthropic/claude-3.5-sonnet"
        ],
        "strategies": [
            "Zero-Shot",
            "Few-Shot",
            "Semantic RAG",
            "Hybrid RAG"
        ],
        "embedding_model": "E5 embedding ONNX (custom wrapper)",
        "vector_store": "FAISS + BM25 + Ensemble",
        "sources_used": [
            "Gramática guaraní.pdf",
            "Diccionario Guaraní-Español / Español-Guaraní.pdf"
        ]
    }
}

# ================================
# 2) GUARDAR JSON DE RESULTADOS
# ================================
with open("guarani_transformation_results.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

print("✓ Resultados guardados en: guarani_transformation_results.json")


# ================================
# 3) EXPORTAR TABLA COMPARATIVA
# ================================
comparison_df.to_csv("comparison_table.csv", index=False)
print("✓ Tabla comparativa guardada: comparison_table.csv")


# ================================
# 4) EXPORTAR VECTOR STORE (FAISS) + ZIP
# ================================
FAISS_DIR = "faiss_store"

import os
os.makedirs(FAISS_DIR, exist_ok=True)

# Guardado en carpeta
vectorstore.save_local(FAISS_DIR)
print(f"✓ Vector store FAISS guardado en carpeta: {FAISS_DIR}")

# Comprimir carpeta a ZIP
zip_name = "faiss_store.zip"
!zip -r {zip_name} {FAISS_DIR} > /dev/null

print(f"✓ Vector store comprimido en ZIP: {zip_name}")


# ================================
# 5) EXPORTAR DOCUMENTOS RAG
# ================================
docs_export = [d.page_content for d in documents]

with open("rag_documents.json", "w", encoding="utf-8") as f:
    json.dump(docs_export, f, ensure_ascii=False, indent=2)

print("✓ Documentos del RAG guardados: rag_documents.json")


# ================================
# 6) EXPORTAR METADATA ADICIONAL
# ================================
metadata_extra = {
    "faiss_zip": zip_name,
    "embedding_model": "E5-ONNX custom wrapper",
    "num_documents": len(documents),
    "chunks_info": {
        "min_length": min(len(d.page_content) for d in documents),
        "max_length": max(len(d.page_content) for d in documents)
    }
}

with open("rag_metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata_extra, f, ensure_ascii=False, indent=2)

print("✓ Metadata adicional guardada: rag_metadata.json")


# ================================
# 7) DESCARGA AUTOMÁTICA (COLAB)
# ================================
try:
    from google.colab import files

    files.download("guarani_transformation_results.json")
    files.download("comparison_table.csv")
    files.download("rag_documents.json")
    files.download("rag_metadata.json")
    files.download(zip_name)  # <-- ZIP DEL VECTOR STORE

    print("✓ Descarga completada")
except Exception as e:
    print("Descarga no disponible:", e)


Guardando resultados...
✓ Resultados guardados en: guarani_transformation_results.json
✓ Tabla comparativa guardada: comparison_table.csv
✓ Vector store FAISS guardado en carpeta: faiss_store
✓ Vector store comprimido en ZIP: faiss_store.zip
✓ Documentos del RAG guardados: rag_documents.json
✓ Metadata adicional guardada: rag_metadata.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Descarga completada


# Resultados y Análisis Comparativo – Transformaciones en Guaraní
---

## Modelos evaluados
- **GPT-3.5 Turbo** (OpenAI)
- **Claude 3.5 Sonnet** (Anthropic)

## Estrategias comparadas
1. **Zero-Shot**
2. **Few-Shot**
3. **Semantic RAG** (FAISS)
4. **Hybrid RAG** (BM25 + FAISS + Few-Shot)

---

# Resultados Obtenidos

| Modelo              | Estrategia      | Accuracy (%) | BLEU  | Correctas / Total |
|---------------------|-----------------|--------------|-------|-------------------|
| GPT-3.5 Turbo       | Zero-Shot       | 0.0          | 59.46 | 0/10              |
| GPT-3.5 Turbo       | Few-Shot        | 10.0         | 0.00  | 1/10              |
| GPT-3.5 Turbo       | Semantic RAG    | 0.0          | 59.46 | 0/10              |
| GPT-3.5 Turbo       | Hybrid RAG      | 10.0         | 0.00  | 1/10              |
| Claude 3.5 Sonnet   | Zero-Shot       | 30.0         | 0.00  | 3/10              |
| Claude 3.5 Sonnet   | Few-Shot        | 50.0         | 0.00  | 5/10              |
| Claude 3.5 Sonnet   | Semantic RAG    | 20.0         | 0.00  | 2/10              |
| Claude 3.5 Sonnet   | Hybrid RAG      | 40.0         | 0.00  | 4/10              |

---

# Mejor Configuración

- **Modelo ganador:** Claude 3.5 Sonnet  
- **Mejor estrategia:** Few-Shot  
- **Accuracy máximo:** **50%**

---

# Análisis del Desempeño

### 1. Claude 3.5 Sonnet supera ampliamente a GPT-3.5 Turbo
Claude demuestra mejores capacidades para comprender y aplicar transformaciones morfológicas en guaraní.

### 2. La estrategia **Few-Shot** es la más efectiva
Agregar ejemplos reales del dataset mejora la capacidad del modelo para seguir patrones lingüísticos.

### 3. **RAG NO mejora el rendimiento**
El contexto recuperado desde la Gramática Guaraní y el Diccionario:
- No contiene transformaciones explícitas **Source → Target**  
- Aporta descripciones teóricas, no reglas aplicables  
- Los chunks son demasiado generales para guiar al LLM  
- Introduce ruido y aumenta la incertidumbre del modelo  

Por eso, las variantes **Semantic RAG** y **Hybrid RAG** no superan al Few-Shot.

---

# Conclusiones

- **Few-Shot > Zero-Shot > RAG**, según nuestros resultados.  
- RAG no ofrece beneficios para este caso porque el conocimiento recuperado no ayuda a realizar transformaciones específicas del dataset.  
- Para mejorar significativamente el rendimiento, el siguiente paso debe ser:  
  **Fine-Tuning supervisado (SFT)** utilizando el *train set* completo.

---

# Archivos generados
- `guarani_transformation_results.json`
- `comparison_table.csv`
- Este archivo: `analysis_results.md`

---
